In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models

In [0]:
df = pd.read_csv('SingleLabel.csv')

In [3]:
df.head()

,artist,genre,title,album,year,lyrics,label
0,Nirvana,Rock,You Know You’re Right,Nirvana,2002.0,I will never bother you\nI will never promise ...,Sadness
1,Damian Marley,Reggae,Here We Go,Stony Hill,2017.0,Here we go\nMy big ego is gonna get me in trou...,Tension
2,The Mission UK,Rock,Jade,Another Fall from Grace,2016.0,She came as Lolita dressed as Venus\nAnd adorn...,Tenderness
3,UB40,Reggae,Food For Thought,Signing Off,1980.0,"Ivory Madonna, dying in the dust\nWaiting for ...",Sadness
4,Johnny Cash,Country,I’ve Been Everywhere,American II: Unchained,1996.0,I was totin' my pack along the dusty Winnemucc...,Sadness


In [74]:
df['label'].unique()

array(['Sadness', 'Tension', 'Tenderness'], dtype=object)

In [0]:
train_size = int(len(df) * .8)

In [0]:
train_cat, test_cat = train_test_split(df['label'], train_size)
train_text, test_text = train_test_split(df['lyrics'], train_size)

In [0]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [0]:
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [0]:
train_cat, test_cat = train_test_split(df['label'], train_size)
train_text, test_text = train_test_split(df['lyrics'], train_size)

In [0]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [0]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [0]:
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [0]:
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [22]:
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (928, 1000)
x_test shape: (232, 1000)
y_train shape: (928, 3)
y_test shape: (232, 3)


In [0]:
def run_experiment(batch_size, epochs, drop_ratio):
  print('batch size: {}, epochs: {}, drop_ratio: {}'.format(
      batch_size, epochs, drop_ratio))
  model = models.Sequential()
  model.add(layers.Dense(512, input_shape=(max_words,)))
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(drop_ratio))
  model.add(layers.Dense(num_classes))
  model.add(layers.Activation('softmax'))

  model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)
  score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=0)
  print('\tTest loss:', score[0])
  print('\tTest accuracy:', score[1])


In [45]:
batch_size = 20
epochs = 3
drop_ratio = 0.3

model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/3
42/42 [==============================] - 0s 6ms/step - loss: 1.0251 - accuracy: 0.5018 - val_loss: 0.9068 - val_accuracy: 0.6344
Epoch 2/3
42/42 [==============================] - 0s 4ms/step - loss: 0.5044 - accuracy: 0.8299 - val_loss: 0.9053 - val_accuracy: 0.6022
Epoch 3/3
42/42 [==============================] - 0s 4ms/step - loss: 0.2802 - accuracy: 0.9353 - val_loss: 0.9828 - val_accuracy: 0.6129


In [46]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

12/12 [==============================] - 0s 2ms/step - loss: 0.9534 - accuracy: 0.6164
Test loss: 0.9533623456954956
Test accuracy: 0.6163793206214905


In [0]:
text_labels = encoder.classes_ 

In [0]:
lol = tokenize.texts_to_matrix(["""
You sound like a bitch, bitch
Shut the fuck up
When your fans become your haters
You done?
Fuck, your beard's weird
Alright
You yellin' at the mic, you weird beard
We doin' this once
Your beard's weird, why you yellin' at the mic?
Rihanna just hit me on a text
Last night I left hickeys on her neck
Wait, you just dissed me? I'm perplexed
Insult me in a line, compliment me on the next, damn
I'm really sorry you want me to have a heart attack
Was watchin' 8 Mile on my NordicTrack
Realized I forgot to call you back
Here's that autograph for your daughter, I wrote it on a Starter cap
Stan, Stan, son, listen, man, dad isn't mad
But how you gonna name yourself after a damn gun and have a man bun?
The giant's woke, eyes open, undeniable
"""])

In [69]:
prediction = model.predict(np.array(lol))
predicted_label = text_labels[np.argmax(prediction)]
print("Predicted label: " + predicted_label + "\n") 

Predicted label: Tension

